In [19]:
import numpy as np
from astropy.time import Time
import random
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from astropy.time import Time
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
# from alertParserUtilities_two import parseAlertContents
import asyncio
import urllib
import json
import lsst.alert.packet as packet
from datetime import datetime
import pandas as pd

In [20]:
def parseAlertContents(dictionary, ra, dec, time, obs_reason, target_name):
    """
    A function to resolve alert contents from the header file of an acquired image
    """
    t = Time.now()
    t_tai = t.tai
    t_mjdtai = t_tai.mjd
    dictionary["diaSource"]["ra"] = ra
    dictionary["diaSource"]["dec"] = dec
    dictionary["observation_reason"] = obs_reason
    dictionary["target_name"] = target_name
    dictionary["diaSource"]["midpointMJDTai"] =  time
    dictionary["diaSource"]["timeProcessedMjdTai"] = time
    
    sourceID = generateSourceID()
    dictionary["diaSourceId"] = sourceID
    dictionary["diaSource"]["diaSourceId"] = sourceID

    return dictionary


def parseAlertContentsFromRecord(dictionary,record,ra,dec):
    """
    A function to resolve alert contents from the header file of an acquired image
    """
    
    dictionary["diaSource"]["ra"] = ra
    dictionary["diaSource"]["dec"] = dec
    dictionary["observation_reason"] = record.observation_reason
    dictionary["target_name"] = record.target_name
    dictionary["diaSource"]["midpointMJDTai"] =  record.timespan.begin.mjd + (record.timespan.end.mjd - record.timespan.begin.mjd) / 2 # This is a 15s offset from the exposure start time 
    dictionary["diaSource"]["timeProcessedMjdTai"] = record.timespan.begin.mjd + 0.00347222 # I add a 5 min offset here
    
    sourceID = generateSourceID()
    dictionary["diaSourceId"] = sourceID
    dictionary["diaSource"]["diaSourceId"] = sourceID

    
    dictionary['diaSource']['time_processed'] = record.timespan.begin.mjd + 0.00347222
    dictionary['diaObject']['validityStart'] = record.timespan.begin.mjd
    dictionary['diaSource']['midpointMjdTai'] = record.timespan.begin.mjd

    return dictionary

def generateSourceID():
    return np.random.randint(9223372036854775808,dtype=np.int64)

def adjustCoordForCenter(center,num=1000):
    floats = np.linspace(0, 1, num=num)
    return np.array(random.choices(floats, k=num))*1.75 + center
    

def getRA(inputList,num=1000):

    raStart = float(getCoordFromKeyword("RASTART",inputList))
    raEnd = float(getCoordFromKeyword("RAEND",inputList))

    avgCenterRA = np.mean([raStart,raEnd]) # Take the average of the two for now
    floats = np.linspace(0, 1, num=num)
    ras = np.array(random.choices(floats, k=num))*1.75 + avgCenterRA

    return ras
    

def getDec(inputList,num=1000):

    decStart = float(getCoordFromKeyword("DECSTART",inputList))
    decEnd = float(getCoordFromKeyword("DECEND",inputList))

    avgCenterDec = np.mean([decStart,decEnd]) # Take the average of the two for now
    floats = np.linspace(0, 1, num=num)
    decs = np.array(random.choices(floats, k=num))*1.75 + avgCenterDec

    return decs

def getTime(inputList):
    midpoint_mjdtai = float(getCoordFromKeyword("MJD-OBS",inputList))
    return midpoint_mjdtai

def getObsReason(inputList):
    obsReasons = getCoordFromKeyword("REASON",inputList,returnAll=True)

    returnString = ""
    for k in obsReasons[3:]:
        returnString+=k
        if k!=obsReasons[-1]:
            returnString+=", "

    return returnString

def getTargetName(inputList):
    
    obsReasons = getCoordFromKeyword("COMMENT",inputList,returnAll=True)

    returnString = ""
    for k in obsReasons[3:]:
        returnString+=k
        if k!=obsReasons[-1]:
            returnString+=", "

    return returnString

def getCoordFromKeyword(keyword,inputList,index=-1,returnAll=False):
    msk = ["keyword: {}".format(keyword) in s for s in inputList]
    firstIndex = np.argwhere(inputList==inputList[msk][0])[0][0]
    if returnAll:
        return inputList[firstIndex+1].split(" ")
    else:
        return inputList[firstIndex+1].split(" ")[index]

In [21]:
from lsst.daf.butler import Butler
butler = Butler("main", instrument="LSSTCam")

In [22]:
records = butler.query_dimension_records('exposure',where="exposure.day_obs = 20250728", limit=1e9)

In [23]:
relRecords = []
for r in records:
    if "too" in r.target_name.lower():
        relRecords.append(r)

In [24]:
outDir = "outAlerts20250728_test2"
os.mkdir(outDir)

FileExistsError: [Errno 17] File exists: 'outAlerts20250728_test2'

In [25]:
schema = packet.SchemaRegistry.from_filesystem(schema_root="lsst.v9_0.alert").get_by_version("9.0")

In [31]:
schema.definition["fields"][1]["default"]=""
schema.definition["fields"][1]["type"]="string"
schema.definition["fields"][2]["default"]=""
schema.definition["fields"][2]["type"]="string"

In [34]:
# schema.definition

In [37]:
# %%time 

myDictionary = {"diaSource":{},"diaObject":{}}
for r in records:
    number=int(7500+np.random.random()*5000) # This will simulate 10k +/- 2.5k alerts
    ras = adjustCoordForCenter(r.tracking_ra,num=number)
    decs = adjustCoordForCenter(r.tracking_dec,num=number)
    bigResult = []
    for k in np.arange(number):
        resultDictionary = parseAlertContentsFromRecord(myDictionary,r, ras[k], decs[k])
        # print(resultDictionary["observation_reason"])
    
        result = packet.simulate.simulate_alert(schema.definition)

    
        for k in resultDictionary.keys():
            if type(result[k])==dict:
                if result[k]==None:
                    result[k]={}
                for k1 in resultDictionary[k].keys():
                    result[k][k1] = resultDictionary[k][k1]
            else:
                result[k] = resultDictionary[k]
    
        bigResult.append(result)
    print(result["observation_reason"],result["target_name"])

    filename=os.path.join(outDir,f"fakeAlert_{r.id}.avro")
    
    with open(filename, "wb") as f:
        schema.store_alerts(f, bigResult)
    print(f"Wrote file to {filename}")

ERROR! Session/line number was not unique in database. History logging moved to new session 1593
tmacheckout UNKNOWN
Wrote file to outAlerts20250728_test2/fakeAlert_2025072800001.avro
tmacheckout UNKNOWN
Wrote file to outAlerts20250728_test2/fakeAlert_2025072800002.avro
tmacheckout UNKNOWN
Wrote file to outAlerts20250728_test2/fakeAlert_2025072800003.avro
infocus_initial_alignment UNKNOWN
Wrote file to outAlerts20250728_test2/fakeAlert_2025072800004.avro
infocus_initial_alignment UNKNOWN
Wrote file to outAlerts20250728_test2/fakeAlert_2025072800005.avro
infocus_initial_alignment UNKNOWN
Wrote file to outAlerts20250728_test2/fakeAlert_2025072800006.avro
infocus_initial_alignment UNKNOWN
Wrote file to outAlerts20250728_test2/fakeAlert_2025072800007.avro
infocus_initial_alignment UNKNOWN
Wrote file to outAlerts20250728_test2/fakeAlert_2025072800008.avro
infocus_initial_alignment UNKNOWN
Wrote file to outAlerts20250728_test2/fakeAlert_2025072800009.avro
infocus_initial_alignment UNKNOWN
Wr

In [36]:
! pwd

/home/s/seanmacb/Rubin Observatory/ToO-analysis-tools/mdc_notebooks
